# Lab 04: Linear Regression and Causality

This lab asks you to do two tasks:

First, to estimate the effect of Right Hearth
Catheterization (RHC)--inserting a tiny hollow tube along the arterial into
the right side of your heart.  It is often done for diagnostical
purposes (it allows to get different measures right in the heart) and
it's usually considered safe.

We will use a dataset about RHC for critically ill patients and see
if RHC is related to increased death rate.  The dataset is downloaded
from [Vanderbilt
Biostats](http://biostat.mc.vanderbilt.edu/wiki/Main/DataSets) and
more information is available at
[http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.html](http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.html).

Second, you have to create (or re-create) the design matrices of the
first problem, analyze their properties, and compute the regression
coefficients of the first part "manually", i.e. by a simple matrix
expression on computer.


## 1 Get Ready

Load the data.  A tab-separated version is available on canvas (files/data).

1. How many observation and variables do we have?
2. The most important variables we use below are _death_ (patient
death recorded/not recorded) and
_swang1_ (rhc performed/not performed).  How are these variables coded?

In [94]:
# Set up
import numpy as np
import pandas as pd
import seaborn as sns # for visualiation
from scipy import stats # ANOVA
from scipy.stats import ttest_ind # t-tests
import statsmodels.formula.api as smf # linear modeling
import matplotlib.pyplot as plt # plotting
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline 

# Load the data, replace strings as numeric
df = pd.read_csv('rhc.csv')
print('number of observations\n',df.size)
df.head()


# Swang1 is an object, in no/ yes with RHC or blanks, Death is yes/no



number of observations
 361305


,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,No,0,...,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,Yes,1,...,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,No,0,...,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,Yes,0,...,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,Yes,0,...,No,No,No,No,No,NaN,64.0,white,Under $11k,11


## 2 Cross-Sectional Estimator

Estimate, using linear regression, how is the RHC related to mortality.

We start easy: let's just find the relationship between recorded death
and rhc.  The important variables are

* _death_: patient death recorded/not recorded
* _swang1_: rhc performed/not performed

This is called "cross-sectional estimator", we just compare
cross-section of individuals who received and did not receive RHC.

Obviously, this is a very crude measure because the hospitals track
patients over different time periods, and if contact is lost early,
the death may not be recorded.  Another obvious problem is that the
patients have very different medical conditions, a factor that
most likely plays a role in the decision whether to perform RHC.


In [95]:
df['swang_num'] = df['swang1'].replace('No RHC', 0).replace('RHC', 1) 
df['death_num'] = df['death'].replace('Yes', 1).replace('No', 0)
lm = smf.ols(formula = 'death_num ~ swang_num', data=df).fit()
lm.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              death_num   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     15.31
Date:                Thu, 19 Apr 2018   Prob (F-statistic):           9.24e-05
Time:                        17:15:51   Log-Likelihood:                -3888.1
No. Observations:                5735   AIC:                             7780.
Df Residuals:                    5733   BIC:                             7794.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6297      0.008     78.709      0.000       0.614       0.645
swang_num      0.0507      0.013      3.912      0.000       0.025       0.076
==============================================================================
Omnibus:                      315.261   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              982.605
Skew:                          -0.622   Prob(JB):                    4.27e-214
Kurtosis:                       1.398   Cond. No.                         2.43
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Comment your results.
##### confidence interval
between 0.025 and 0.076
##### Betas: 
Intercept 0.6297, slope is 0.0507-> lies within our confidence interval 
R squared in low, like 2% not so good

## 3 Address some of the issues

Now let's try to address some of the issues with the estimator above.
The dataset includes many other patient descriptors.


### 3.1 Let's include 'age' and 'sex'.
#### How are these coded?

Sex is in male and female
Age is a numeric value

In [96]:
df['sex_num'] = df['sex'].replace('Male', 0).replace('Female', 1) 
print('data type and sex is ', df['sex'].dtype)
print('data type of age is ', df['age'].dtype)


data type and sex is  object
data type of age is  float64


#### What do you find?
Age and sex should be added into the regression. May increse R squared value




### 3.2 Include age and sex in the regression

Now allow the death to depend on gender and age, on top of that it may
depend on rhc.  Note that it may not just depend on age in a linear
fashion but in a much more complex way, so include not just $age$ but
also $age^2$ and $age^3$ as explanatory variables.

In [97]:
df['age2'] = df['age'] * df['age']
df['age3'] = df['age'] * df['age'] * df['age']
lm1 = smf.ols(formula = 'death_num ~ swang_num + sex_num + age', data=df).fit()
lm2 = smf.ols(formula = 'death_num ~ swang_num + sex_num + age2', data=df).fit()
lm3 = smf.ols(formula = 'death_num ~ swang_num + sex_num  + age3', data=df).fit()
lm1.summary()
lm2.summary()
lm3.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              death_num   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     88.03
Date:                Thu, 19 Apr 2018   Prob (F-statistic):           1.09e-55
Time:                        17:15:55   Log-Likelihood:                -3766.6
No. Observations:                5735   AIC:                             7541.
Df Residuals:                    5731   BIC:                             7568.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4903      0.013     36.360      0.000       0.464       0.517
swang_num      0.0610      0.013      4.795      0.000       0.036       0.086
sex_num       -0.0264      0.012     -2.122      0.034      -0.051      -0.002
age3        5.251e-07   3.35e-08     15.696      0.000     4.6e-07    5.91e-07
==============================================================================
Omnibus:                      281.054   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              843.800
Skew:                          -0.582   Prob(JB):                    5.90e-184
Kurtosis:                       1.525   Cond. No.                     8.96e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.96e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Comment your results
#### Betas: 
Intercept is 0.2561, slope of swang/death is 0.0559, the slope of sex/death is -0.0238 (not sure if men or woomen were baseline) and the slope of age/death is 0.0062

#### Confidence: 
There is a 95% chance that the swang outcome will lie between -0.031 and 0.081 of the intercept. Sex is between -0.048 and	0.001 of the intercept, and age is -0.006 and 0.007 of the intercept. looks like sometthing odd is going on with age

#### R Squared: 5%. not so good



## 4 Design matrices of the models

Each linear model (and many other models) have associated _design
matrices_.  Design matrix is the matrix of all your explanatory
variables (all x-s) in the final numeric matrix form.  This includes
* adding a constant column
* converting categorical/non-numeric variables into suitable numeric
variables

You next task is to create/extract the design matrices of both of the
models above, investigate their numeric properties (condition
numbers), and solve the linear regression problem in matrix form.

We did not have time in the class to talk about it, but there is a
closed-form solution for the linear regression problem:  beta =
$(X'X)^{-1} X'y$.  Compute this solution and compare with the regression
package output above.


### 4.1 First model

#### 4.1.1 create the design matrix of it, X.

Depending on the way you solved your problem, you may already have
created it.  Depending on the way you solved the problem above, you
may be able to extract it from the existing model.  You may also redo it
manually here.  Remember:
* include the constant term!
* design matrix must be a _matrix_, not data frame or something else.

In [98]:
import patsy
f = 'death_num ~ swang_num'
y,X = patsy.dmatrices(f, df, return_type='matrix')
print('death matrix\n', y)
print('swang matrix\n', X)


death matrix
 [[0.]
 [1.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]
swang matrix
 [[1. 0.]
 [1. 1.]
 [1. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [101]:
h = 'death_num ~ swang_num + sex_num + age2'
y_1,X_1 = patsy.dmatrices(h, df, return_type='matrix')
print('death matrix\n', y_1)
print('swang, age, sex matrix\n', X_1)


death matrix
 [[0.]
 [1.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]
swang, age, sex matrix
 [[1.00000000e+00 0.00000000e+00 0.00000000e+00 4.93520019e+03]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 6.11194979e+03]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 2.12447062e+03]
 ...
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 6.47783362e+03]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 4.53992560e+03]
 [1.00000000e+00 0.00000000e+00 1.00000000e+00 2.98814962e+03]]


#### 4.1.2 Compute the condition number of X`X.

You may choose whatever definition you like, but please report what
are you using.

In [102]:
# using Matrix definition using sex, age squared, and swang
condition_number = X_1.T @ X
print('Condition number\n', condition_number)

Condition number
 [[5.73500000e+03 2.18400000e+03]
 [2.18400000e+03 2.18400000e+03]
 [2.54300000e+03 9.06000000e+02]
 [2.31994357e+07 8.59349270e+06]]


#### 4.1.3 Compute your regression coefficients using the formula above.

Note: you also need your outcome variable $y$ in numeric matrix
form. 

In [103]:
sm.OLS(y_1, X_1).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              death_num   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     95.89
Date:                Thu, 19 Apr 2018   Prob (F-statistic):           1.48e-60
Time:                        17:16:45   Log-Likelihood:                -3755.3
No. Observations:                5735   AIC:                             7519.
Df Residuals:                    5731   BIC:                             7545.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4270      0.016     26.600      0.000       0.396       0.458
swang_num      0.0589      0.013      4.636      0.000       0.034       0.084
sex_num       -0.0253      0.012     -2.042      0.041      -0.050      -0.001
age2        5.212e-05   3.17e-06     16.428      0.000    4.59e-05    5.83e-05
==============================================================================
Omnibus:                      282.228   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              829.038
Skew:                          -0.583   Prob(JB):                    9.48e-181
Kurtosis:                       1.548   Cond. No.                     1.30e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.3e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### 4.1.4 Compare your coefficients here with the OLS results above.

In [ ]:
# they are exactly the same betas. GO US :) 